# ML/ Image Quality Detector
> part 6

### Optimization

1. optimize the learning 
2. expand the dataset up till 1000 images
3. increase the  filters up to 64
4. increase the number of training epochh 5000-10000
5. optimize on GPU
https://pytorch.org/get-started/locally/
    for  CUDA or ROCm
    -  install the drivre CUDA
    -  install the torch with CUDA suport
    -  check & link to device
    - !!! transfer all tensors & network to device
6. optimize the stop conditions based on accuracy
7. resource consumption


# HW
1. expand the dataset up till 2000 images
2. increase the  filters up to 96 &  kernel size / stride = 10x10
3. increase the number of training epochh 5000-10000 |acc > 99.90

In [2]:
import os

from PIL import Image,ImageOps
import matplotlib.pyplot as plt
import numpy as np 
import torch, torch.nn as nn, torch.optim as optim


# Transfer on GPU with CUDA 
# print("GPU avaible ?",torch.cuda.is_available())
# print("Device",torch.cuda.current_device())
                            # V---- device NR
# device = torch.device("cuda:0")


In [7]:
# Image Helpers 
def loadImages(folder,start_number= 0,end_number= 2000):
    images = []
    count = 0
    file_names = os.listdir(folder)
    for number in range(start_number,end_number):
            file_name= f"image_{number:05d}.jpg"
            img = Image.open(folder+"/"+file_name)
            img = ImageOps.grayscale(img)
            img_matrix= np.array(img)
            images.append(img_matrix)
            # print(file_name)
            count += 1
            if count == number:
                  break

            
    return images

def imageToTensor(img):
      img_tensor = torch.from_numpy(img).type(torch.FloatTensor).view(1,500,500)
      return img_tensor

In [4]:
## Training helpers
## Let's test it
def estimateAccuracy(model,x,y):
    correct = 0 
    for i in range(len(x)):
        x_tensor= imageToTensor(x[i])     #.to(device)        # for CUDA device transfer
        y_predicted = model(x_tensor)
        y_tensor = torch.FloatTensor(y[i])#.to(device)        # for CUDA device transfer
        if(torch.argmax(y_predicted)) == torch.argmax(y_tensor):
            correct += 1

            
    accuracy = (correct / len(x))* 100       
    print(f"predicted correctly {accuracy}%")
    return accuracy

def oneEpochTrain(model,x,y,criterion,optimizer):
    for i in range(len(x)):

        x_tensor= imageToTensor(x[i])#.to(device)        # for CUDA device transfer
        y_tensor = torch.FloatTensor(y[i])#.to(device)   # for CUDA device transfer

        optimizer.zero_grad()
        
        y_predicted = model(x_tensor)

        loss = criterion(y_predicted, y_tensor)

        loss.backward()
        optimizer.step()

    # if epoch % 10 == 0:
        # print(f"epoch = {epoch:5},loss= {loss:10.8f}")


In [11]:
NUMBER_OF_DATA_SAMPLES =1000

# PREPARING DATASET
original_images  = loadImages("images/original")
# blurred_images   = loadImages("images/blurred")   
lowq_images      = loadImages("images/lowq")      
# sharpened_images = loadImages("images/sharpened")
print(f"loaded{len(original_images)+ len(lowq_images)} images")


## Preparing Data sets
x = original_images + lowq_images # 100 [50 - good, 50 - bad]
#      V --- GOOD                    V-- BAD 
y = [[0,1] for _ in range(NUMBER_OF_DATA_SAMPLES)] + [[1,0] for _ in range(NUMBER_OF_DATA_SAMPLES)]

loaded100 images


In [ ]:
# Prep The Model     -->  create multi layer of neurons NN
model = nn.Sequential(
    #                            v--nn.layers      V-- pixels taken         V -- pixels steps
    nn.Conv2d(in_channels=1,out_channels=64,kernel_size=(20,20), stride = (20,20)), # layer 0 <- Neurons
    nn.Flatten(start_dim=0), # <------ No neurons  / transform in flatenn function
    nn.Linear(in_features=40000, out_features=2),                                    # layer 1 <- Neurons
)
# for CUDA device transfer
# model.to(device)


criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(),lr=0.00001)

In [ ]:
## Training setup
for epoch in range(5000):
    oneEpochTrain(model,x,y,criterion,optimizer)
    print(f"epoch:{epoch:5}")
    accuracy = estimateAccuracy(model,x,y)    
    if accuracy >= 99.90:
        break

print("TRANING FINISHED!!!")

epoch:    0
predicted correctly 50.0%
epoch:    1
predicted correctly 50.0%
epoch:    2
predicted correctly 50.0%
epoch:    3
predicted correctly 50.0%
epoch:    4
predicted correctly 50.0%
epoch:    5
predicted correctly 50.0%
epoch:    6
predicted correctly 50.0%
epoch:    7
predicted correctly 50.0%
epoch:    8
predicted correctly 50.0%
epoch:    9
predicted correctly 50.0%
epoch:   10
predicted correctly 50.0%
epoch:   11
predicted correctly 50.0%
epoch:   12
predicted correctly 50.0%
epoch:   13
predicted correctly 50.1%
epoch:   14
predicted correctly 50.1%
epoch:   15
predicted correctly 50.1%
epoch:   16
predicted correctly 50.1%
epoch:   17
predicted correctly 50.2%
epoch:   18
predicted correctly 50.2%
epoch:   19
predicted correctly 50.3%
epoch:   20
predicted correctly 50.2%
epoch:   21
predicted correctly 50.4%
epoch:   22
predicted correctly 50.3%
epoch:   23
predicted correctly 50.3%
epoch:   24
predicted correctly 50.3%
epoch:   25
predicted correctly 50.2%
epoch:   26


In [ ]:
print(model.state_dict())

In [ ]:
## Save the trained network <--https://pytorch.org/tutorials/beginner/saving_loading_models.html
torch.save(model.state_dict(),'models/image-quality-detector-64-20x20-1000-x-images')

In [ ]:
## Load the trained network <--https://pytorch.org/tutorials/beginner/saving_loading_models.html
# model = TheModelClass(*args, **kwargs) # <--recreate the model object
model.load_state_dict(torch.load('models/image-quality-detector-64-20x20-1000-x-images'))
model.eval()

Sequential(
  (0): Conv2d(1, 64, kernel_size=(20, 20), stride=(20, 20))
  (1): Flatten(start_dim=0, end_dim=-1)
  (2): Linear(in_features=40000, out_features=2, bias=True)
)

In [ ]:
# PREPARING TEST DATASET
NUMBER_OF_DATA_SAMPLES =500

original_images  = loadImages("images/original", start_number = 500, end_number=1000)
lowq_images      = loadImages("images/lowq",     start_number = 500, end_number=1000)      
# blurred_images   = loadImages("images/blurred")   
# sharpened_images = loadImages("images/sharpened")
print(f"loaded{len(original_images)+ len(lowq_images)} images")


## Preparing Data sets
x_test = original_images + lowq_images # 100 [50 - good, 50 - bad]
#      V --- GOOD                    V-- BAD 
y_test = [[0,1] for _ in range(NUMBER_OF_DATA_SAMPLES)] + [[1,0] for _ in range(NUMBER_OF_DATA_SAMPLES)]

NameError: name 'loadImages' is not defined

## TEST the network

In [ ]:
estimateAccuracy(model,x_test,y_test)

predicted correctly 73.9%


73.9

In [ ]:
## HW*
# 1. download more than 1500+
# 2. wrap the code bellow in one more loop 
# 3. remove either manually,  or using os.remove()

## detect wich images are identic with a selected one 
for n in range(0,99):
    number1 = n
    file_name = f"image_{number1:05d}.jpg"
    file1 = open("images/sharpened"+"/"+file_name, "rb")
    content_1 = file1.read() 
    file1.close()

    for number in range(0,99):
        
        file_name = f"image_{number:05d}.jpg"
        file1 = open("images/sharpened"+"/"+file_name, "rb")
        content = file1.read() 
        file1.close()

        if content_1 == content:
            print(f"{number1} identic {number}")

      
        


        

0 identic 0
1 identic 1
2 identic 2
3 identic 3
4 identic 4
5 identic 5
5 identic 25
6 identic 6
7 identic 7
8 identic 8
9 identic 9
10 identic 10
11 identic 11
11 identic 35
12 identic 12
13 identic 13
14 identic 14
15 identic 15
16 identic 16
17 identic 17
18 identic 18
19 identic 19
20 identic 20
21 identic 21
22 identic 22
23 identic 23
24 identic 24
25 identic 5
25 identic 25
26 identic 26
27 identic 27
28 identic 28
29 identic 29
30 identic 30
31 identic 31
32 identic 32
33 identic 33
34 identic 34
35 identic 11
35 identic 35
36 identic 36
37 identic 37
38 identic 38
39 identic 39
40 identic 40
41 identic 41
42 identic 42
43 identic 43
44 identic 44
45 identic 45
46 identic 46
47 identic 47
48 identic 48
49 identic 49
50 identic 50
51 identic 51
52 identic 52
53 identic 53
54 identic 54
55 identic 55
56 identic 56
57 identic 57
58 identic 58
59 identic 59
60 identic 60
61 identic 61
62 identic 62
63 identic 63
64 identic 64
65 identic 65
66 identic 66
67 identic 67
68 identic 68
